# Meteo Bronze Pipeline

This notebook handles the *Bronze* stage of the Meteo data pipeline.  
It is responsible for fetching **raw weather data** from the [Open-Meteo API](https://open-meteo.com/en/docs)  
and saving it locally for later cleaning and processing in the Silver stage.

---

### 📋 Objectives
1. Define reference data for Norwegian price areas and their representative cities.
2. Create a function to fetch weather data from Open-Meteo’s ERA5 reanalysis model.
3. Download and store raw JSON/CSV responses for each city and year.

---

**Outputs:**
- `data/bronze/df_cities.csv` – city reference table  
- `data/bronze/meteo_<city>_<year>.json` – raw downloaded data


In [1]:
import pandas as pd
from pathlib import Path
import requests
import json

# Set up paths
DATA_DIR = Path("../../data/bronze")
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Data folder ready: {DATA_DIR.resolve()}")

✅ Data folder ready: /Users/fabianheflo/UNI_courses/IND320/IND320/data/bronze


## Task 1 — Define price areas and representative cities

Create a reference table for the five Norwegian price areas (NO1–NO5),  
their representative cities, and corresponding latitude/longitude coordinates.
This table will be used in later tasks to request weather data from Open-Meteo.

In [2]:
# Define cities and price areas
cities = [
    {"price_area": "NO1", "city": "Oslo", "latitude": 59.9139, "longitude": 10.7522},
    {"price_area": "NO2", "city": "Kristiansand", "latitude": 58.1467, "longitude": 7.9956},
    {"price_area": "NO3", "city": "Trondheim", "latitude": 63.4305, "longitude": 10.3951},
    {"price_area": "NO4", "city": "Tromsø", "latitude": 69.6492, "longitude": 18.9553},
    {"price_area": "NO5", "city": "Bergen", "latitude": 60.3913, "longitude": 5.3221},
]

df_cities = pd.DataFrame(cities)
display(df_cities)

# Save to bronze folder
path = DATA_DIR / "df_cities.csv"
df_cities.to_csv(path, index=False)
print(f"💾 Saved city reference table → {path}")

,price_area,city,latitude,longitude
0,NO1,Oslo,59.9139,10.7522
1,NO2,Kristiansand,58.1467,7.9956
2,NO3,Trondheim,63.4305,10.3951
3,NO4,Tromsø,69.6492,18.9553
4,NO5,Bergen,60.3913,5.3221


💾 Saved city reference table → ../../data/bronze/df_cities.csv


In [3]:
# Verify file was saved correctly
pd.read_csv(DATA_DIR / "df_cities.csv").head()

,price_area,city,latitude,longitude
0,NO1,Oslo,59.9139,10.7522
1,NO2,Kristiansand,58.1467,7.9956
2,NO3,Trondheim,63.4305,10.3951
3,NO4,Tromsø,69.6492,18.9553
4,NO5,Bergen,60.3913,5.3221
